In [1]:
import os
import logging
import re
import pickle
from typing import List, Optional

import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from tqdm.notebook import tqdm

In [2]:
# Создаём внутреннюю папку проекта
os.makedirs('pipeline', exist_ok=True) 

In [16]:
"""
Переключатель логирования функций пайплайна.
При выборе опции'INFO' будут выводиться названия функций, 
названия исходных обрабатываемых  признаков 
и названия новых фичей.
В классификаторе будут выводиться названия методов,
этапы обучения ансамбля и гиперпараметры моделей ансамбля.
При выборе опции 'OFF' логи отключаются.
"""

log_level_input = input(
    """
Введите уровень логирования для pipeline функций
'INFO' для включения, 'OFF' для отключения
"""
).strip().upper()

if log_level_input == 'OFF':
    # Блокируем логи
    logging.disable(logging.CRITICAL)  
    print("Логирование pipeline функций отключено")

elif log_level_input == 'INFO':
    # Снимаем блокировку
    logging.disable(logging.NOTSET) 

    # Удаляем старые обработчики, чтобы basicConfig сработал
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(message)s'
    )
    print("Логирование pipeline функций включено")

else:
    # Снимаем блокировку
    logging.disable(logging.NOTSET)  

    # Удаляем старые обработчики, чтобы basicConfig сработал
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(message)s'
    )
    print("Неверный ввод, установлен режим INFO")

# Создаём объект логера
logger = logging.getLogger(__name__)


Введите уровень логирования для pipeline функций
'INFO' для включения, 'OFF' для отключения
 off


Логирование pipeline функций отключено


# Lists of features for the functions

## Basic lists

In [3]:
# rn - уникальный признак

# Бинаризированные
pre_features = [
    'pre_since_opened',
    'pre_since_confirmed',
    'pre_pterm',
    'pre_fterm',
    'pre_till_pclose',
    'pre_till_fclose',
    'pre_loans_credit_limit',
    'pre_loans_next_pay_summ',
    'pre_loans_outstanding',
    'pre_loans_max_overdue_sum',
    'pre_loans_credit_cost_rate',
    'pre_loans5',
    'pre_loans530',
    'pre_loans3060',
    'pre_loans6090',
    'pre_loans90',
    'pre_util',
    'pre_over2limit',
    'pre_maxover2limit'
]

# Закодированные
enc_features = [
    'enc_loans_account_holder_type',
    'enc_loans_credit_status',
    'enc_loans_credit_type',
    'enc_loans_account_cur'
]

# Статусы ежемесячных платежей
enc_paym_features = [
    'enc_paym_0',
    'enc_paym_1',
    'enc_paym_2',
    'enc_paym_3',
    'enc_paym_4',
    'enc_paym_5',
    'enc_paym_6',
    'enc_paym_7',
    'enc_paym_8',
    'enc_paym_9',
    'enc_paym_10',
    'enc_paym_11',
    'enc_paym_12',
    'enc_paym_13',
    'enc_paym_14',
    'enc_paym_15',
    'enc_paym_16',
    'enc_paym_17',
    'enc_paym_18',
    'enc_paym_19',
    'enc_paym_20',
    'enc_paym_21',
    'enc_paym_22',
    'enc_paym_23',
    'enc_paym_24'
]

#  Флаги
flag_features = [
    'is_zero_loans5',
    'is_zero_loans530',
    'is_zero_loans3060',
    'is_zero_loans6090',
    'is_zero_loans90',
    'is_zero_util',
    'is_zero_over2limit',
    'is_zero_maxover2limit',
    'pclose_flag',
    'fclose_flag'
]

In [4]:
df_source = pd.read_csv('prepared_data/source_data_train_1.csv')
df_source.shape

(20931476, 61)

In [5]:
df_result = pd.read_csv('prepared_data/cut_corr_imp_train.csv')
df_result.shape

(2400000, 61)

In [6]:
df_source_columns = df_source.columns.tolist()
df_source_columns[:10]

['id',
 'rn',
 'pre_since_opened',
 'pre_since_confirmed',
 'pre_pterm',
 'pre_fterm',
 'pre_till_pclose',
 'pre_till_fclose',
 'pre_loans_credit_limit',
 'pre_loans_next_pay_summ']

In [7]:
df_result_columns = df_result.columns.tolist()
df_result_columns[:10]

['id',
 'flag',
 'is_zero_sum_prop_1',
 'enc_paym_avg_0_1_this_year_diff',
 'pre_util_prop_3',
 'enc_loans_credit_type_prop_0',
 'pre_till_pclose_prop_10',
 'pre_util_prop_6',
 'pre_loans_outstanding_prop_1',
 'pre_util_mean_freq']

## List of features to download from the original dataset

In [8]:
"""
Формируем список колонок из df_source_columns,
которые НЕ встречаются ни в одном названии из df_result_columns как подстрока.
"""
drop_list = []
for col_source in df_source_columns:
    found = False
    for col_result in df_result_columns:
        if col_source in col_result:
            found = True
            break
    if not found:
        drop_list.append(col_source)

print(len(drop_list))
drop_list

30


['pre_loans_total_overdue',
 'pre_loans3060',
 'pre_loans6090',
 'pre_loans90',
 'is_zero_loans3060',
 'is_zero_loans6090',
 'is_zero_loans90',
 'pre_maxover2limit',
 'is_zero_util',
 'is_zero_maxover2limit',
 'enc_paym_3',
 'enc_paym_4',
 'enc_paym_5',
 'enc_paym_6',
 'enc_paym_7',
 'enc_paym_11',
 'enc_paym_12',
 'enc_paym_13',
 'enc_paym_14',
 'enc_paym_15',
 'enc_paym_16',
 'enc_paym_17',
 'enc_paym_18',
 'enc_paym_19',
 'enc_paym_20',
 'enc_paym_21',
 'enc_paym_22',
 'enc_paym_23',
 'pclose_flag',
 'fclose_flag']

In [9]:
needed_columns = [x for x in df_source_columns if x not in drop_list]

print(len(needed_columns))
needed_columns[:10]

31


['id',
 'rn',
 'pre_since_opened',
 'pre_since_confirmed',
 'pre_pterm',
 'pre_fterm',
 'pre_till_pclose',
 'pre_till_fclose',
 'pre_loans_credit_limit',
 'pre_loans_next_pay_summ']

In [10]:
"""
Добавим недостающие признаки из групп flag_features и enc_paym _features, 
для правильной работы функций обрабатывающих эти группы. 
"""
features_list= [
    'is_zero_loans3060',
    'is_zero_loans6090',
    'is_zero_loans90',
    'enc_paym_3',
    'enc_paym_4',
    'enc_paym_5',
    'enc_paym_6',
    'enc_paym_7',
    'enc_paym_11',
    'enc_paym_12',
    'enc_paym_13',
    'enc_paym_14',
    'enc_paym_15',
    'enc_paym_16',
    'enc_paym_17',
    'enc_paym_18',
    'enc_paym_19',
    'enc_paym_20',
    'enc_paym_21',
    'enc_paym_22',
    'enc_paym_23'
]

# Список признаков для скачивания из исходного датасета
needed_columns = needed_columns + features_list

print(len(needed_columns))
needed_columns[:10]

52


['id',
 'rn',
 'pre_since_opened',
 'pre_since_confirmed',
 'pre_pterm',
 'pre_fterm',
 'pre_till_pclose',
 'pre_till_fclose',
 'pre_loans_credit_limit',
 'pre_loans_next_pay_summ']

## Create_definite_value_proportion_features_pipeline funtion list

In [11]:
# Создадим список пропорциональных фичей в итоговом датасете
prop_features_result_list = [col for col in df_result_columns if 'prop_' in col]

print(len(prop_features_result_list))
prop_features_result_list

38


['is_zero_sum_prop_1',
 'pre_util_prop_3',
 'enc_loans_credit_type_prop_0',
 'pre_till_pclose_prop_10',
 'pre_util_prop_6',
 'pre_loans_outstanding_prop_1',
 'pre_loans_credit_limit_prop_2',
 'pre_loans_credit_cost_rate_prop_6',
 'pre_loans_outstanding_prop_5',
 'pre_loans_credit_cost_rate_prop_11',
 'pre_loans_credit_cost_rate_prop_4',
 'pre_loans_next_pay_summ_prop_5',
 'pre_since_opened_prop_12',
 'pre_loans_credit_limit_prop_15',
 'enc_loans_credit_type_prop_2',
 'pre_fterm_prop_7',
 'enc_paym_0_prop_1',
 'is_zero_over2limit_prop_1',
 'pre_since_opened_prop_8',
 'pre_loans_max_overdue_sum_prop_1',
 'pre_loans_next_pay_summ_prop_0',
 'pre_pterm_prop_6',
 'pre_since_opened_prop_19',
 'is_zero_loans5_prop_1',
 'enc_loans_account_holder_type_prop_4',
 'pre_loans_credit_limit_prop_18',
 'pre_till_fclose_prop_4',
 'pre_pterm_prop_3',
 'is_zero_loans530_prop_1',
 'enc_loans_credit_status_prop_5',
 'pre_since_confirmed_prop_4',
 'pre_fterm_prop_3',
 'pre_till_fclose_prop_3',
 'pre_till_fcl

In [12]:
# Создадим список признаков исходного датасета из которых были сделаны пропорциональные фичи
prop_features_source_list = list(
    set(
        [
            re.sub(r'_prop.*$', '', col)
            for col in prop_features_result_list
        ]
    )
)

print(len(prop_features_source_list))
prop_features_source_list

22


['is_zero_over2limit',
 'pre_till_fclose',
 'is_zero_sum',
 'pre_loans_credit_cost_rate',
 'enc_paym_24',
 'pre_loans_next_pay_summ',
 'pre_till_pclose',
 'pre_loans_outstanding',
 'pre_since_confirmed',
 'pre_fterm',
 'pre_util',
 'pre_loans_credit_limit',
 'pre_since_opened',
 'pre_loans_max_overdue_sum',
 'enc_loans_credit_type',
 'is_zero_loans5',
 'enc_loans_account_holder_type',
 'pre_over2limit',
 'pre_pterm',
 'enc_loans_credit_status',
 'enc_paym_0',
 'is_zero_loans530']

In [13]:
# Соберем часть словаря пропорциональных фичей для пайплайна
prop_features_dict = {}

for source_col in prop_features_source_list:
    # Инициализируем пустой список для каждого исходного признака
    prop_features_dict[source_col] = []
    # Создадим паттерн: имя col в начале и после него подчёркивание или конец строки
    pattern = re.compile(r'^' + source_col + r'(_|$)')
    for result_col in prop_features_result_list:
        # Проверяем, совпадает ли имя признака с паттерном
        if pattern.match(result_col):
            # Ищем число в конце строки
            match = re.search(r'(\d+)$', result_col)
            # Добавляем найденное число в список для данного source_col
            prop_features_dict[source_col].append(int(match.group(1)))
prop_features_dict

{'is_zero_over2limit': [1],
 'pre_till_fclose': [4, 3, 1],
 'is_zero_sum': [1],
 'pre_loans_credit_cost_rate': [6, 11, 4],
 'enc_paym_24': [1],
 'pre_loans_next_pay_summ': [5, 0],
 'pre_till_pclose': [10, 7],
 'pre_loans_outstanding': [1, 5],
 'pre_since_confirmed': [4, 7],
 'pre_fterm': [7, 3],
 'pre_util': [3, 6],
 'pre_loans_credit_limit': [2, 15, 18],
 'pre_since_opened': [12, 8, 19],
 'pre_loans_max_overdue_sum': [1],
 'enc_loans_credit_type': [0, 2],
 'is_zero_loans5': [1],
 'enc_loans_account_holder_type': [4],
 'pre_over2limit': [17],
 'pre_pterm': [6, 3],
 'enc_loans_credit_status': [5],
 'enc_paym_0': [1],
 'is_zero_loans530': [1]}

In [14]:
"""
Добавим в словарь недостающие is_zero_loans* для функции суммирования.
Удалим is_zero_sum, фича is_zero_sum_prop_1 будет собираться другой функцией.
"""
is_zero_loans_list = [
        'is_zero_loans5',
        'is_zero_loans530',
        'is_zero_loans3060',
        'is_zero_loans6090',
        'is_zero_loans90'
    ]
for col in is_zero_loans_list:
    if col not in prop_features_dict.keys():
        prop_features_dict[col] = [1]
        
del prop_features_dict['is_zero_sum']

prop_features_dict

{'is_zero_over2limit': [1],
 'pre_till_fclose': [4, 3, 1],
 'pre_loans_credit_cost_rate': [6, 11, 4],
 'enc_paym_24': [1],
 'pre_loans_next_pay_summ': [5, 0],
 'pre_till_pclose': [10, 7],
 'pre_loans_outstanding': [1, 5],
 'pre_since_confirmed': [4, 7],
 'pre_fterm': [7, 3],
 'pre_util': [3, 6],
 'pre_loans_credit_limit': [2, 15, 18],
 'pre_since_opened': [12, 8, 19],
 'pre_loans_max_overdue_sum': [1],
 'enc_loans_credit_type': [0, 2],
 'is_zero_loans5': [1],
 'enc_loans_account_holder_type': [4],
 'pre_over2limit': [17],
 'pre_pterm': [6, 3],
 'enc_loans_credit_status': [5],
 'enc_paym_0': [1],
 'is_zero_loans530': [1],
 'is_zero_loans3060': [1],
 'is_zero_loans6090': [1],
 'is_zero_loans90': [1]}

## List for create_mean_value_frequency_feature_pipeline features

In [15]:
# Соберем список всех фичей средней частотности в итоговом датасете
mean_freq_result_list = [col for col in df_result_columns if 'mean_freq' in col]

print(len(mean_freq_result_list))
mean_freq_result_list

16


['pre_util_mean_freq',
 'pre_loans_credit_limit_mean_freq',
 'pre_since_opened_mean_freq',
 'pre_loans_credit_cost_rate_mean_freq',
 'enc_loans_credit_type_mean_freq',
 'pre_loans_next_pay_summ_mean_freq',
 'pre_since_confirmed_mean_freq',
 'pre_pterm_mean_freq',
 'enc_paym_0_mean_freq',
 'enc_loans_account_holder_type_mean_freq',
 'pre_loans530_mean_freq',
 'enc_paym_8_mean_freq',
 'pre_loans5_mean_freq',
 'enc_paym_10_mean_freq',
 'enc_loans_account_cur_mean_freq',
 'enc_paym_9_mean_freq']

In [16]:
"""
Соберем список признаков исходного датасета 
из которых были сделаны фичи средней частотности.
"""
mean_freq_source_list = [x[:-len('_mean_freq')] for x in mean_freq_result_list]
print(len(mean_freq_source_list))
mean_freq_source_list

16


['pre_util',
 'pre_loans_credit_limit',
 'pre_since_opened',
 'pre_loans_credit_cost_rate',
 'enc_loans_credit_type',
 'pre_loans_next_pay_summ',
 'pre_since_confirmed',
 'pre_pterm',
 'enc_paym_0',
 'enc_loans_account_holder_type',
 'pre_loans530',
 'enc_paym_8',
 'pre_loans5',
 'enc_paym_10',
 'enc_loans_account_cur',
 'enc_paym_9']

## Drop list

In [17]:
temporary_features_list = [
    'enc_paym_avg_1_all',
    'enc_paym_avg_2_all',
    'enc_paym_avg_0_this_year',
    'enc_paym_avg_1_this_year',
    'enc_paym_avg_0_last_year',
    'is_zero_loans3060_prop_1',
    'is_zero_loans6090_prop_1',
    'is_zero_loans90_prop_1'
]

In [18]:
drop_list = needed_columns + temporary_features_list
print(len(drop_list))
drop_list

60


['id',
 'rn',
 'pre_since_opened',
 'pre_since_confirmed',
 'pre_pterm',
 'pre_fterm',
 'pre_till_pclose',
 'pre_till_fclose',
 'pre_loans_credit_limit',
 'pre_loans_next_pay_summ',
 'pre_loans_outstanding',
 'pre_loans_max_overdue_sum',
 'pre_loans_credit_cost_rate',
 'pre_loans5',
 'pre_loans530',
 'is_zero_loans5',
 'is_zero_loans530',
 'pre_util',
 'pre_over2limit',
 'is_zero_over2limit',
 'enc_paym_0',
 'enc_paym_1',
 'enc_paym_2',
 'enc_paym_8',
 'enc_paym_9',
 'enc_paym_10',
 'enc_paym_24',
 'enc_loans_account_holder_type',
 'enc_loans_credit_status',
 'enc_loans_credit_type',
 'enc_loans_account_cur',
 'is_zero_loans3060',
 'is_zero_loans6090',
 'is_zero_loans90',
 'enc_paym_3',
 'enc_paym_4',
 'enc_paym_5',
 'enc_paym_6',
 'enc_paym_7',
 'enc_paym_11',
 'enc_paym_12',
 'enc_paym_13',
 'enc_paym_14',
 'enc_paym_15',
 'enc_paym_16',
 'enc_paym_17',
 'enc_paym_18',
 'enc_paym_19',
 'enc_paym_20',
 'enc_paym_21',
 'enc_paym_22',
 'enc_paym_23',
 'enc_paym_avg_1_all',
 'enc_paym_av

# Downloading dataset and target

In [19]:
# СКАЧИВАЕМ ИСХОДНЫЙ ДАТАСЕТ

def read_parquet_dataset_from_local(
    path_to_dataset: str,
    start_from: int = 0,
    num_parts_to_read: int = 2,
    columns: Optional[List[str]] = None,
    verbose: bool = False
) -> pd.DataFrame:
    """
    Читает num_parts_to_read партиций, преобразовывает их к pd.DataFrame и возвращает.

    Args:
        path_to_dataset : путь до директории с партициями
        start_from : номер партиции, с которой нужно начать чтение
        num_parts_to_read : количество партиций, которые требуется прочитать
        columns : список колонок, которые нужно прочитать из партиции
        verbose : выводить ли дополнительную информацию
    
    Returns:
        pd.DataFrame 
    """
    res = []
    dataset_paths = sorted(
        os.path.join(path_to_dataset, filename)
        for filename in os.listdir(path_to_dataset)
        if filename.startswith('train')
    )

    if verbose:
        print('Dataset paths:')
        for path in dataset_paths:
            print(path)

    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]

    if verbose:
        print('Reading chunks:')
        for chunk in chunks:
            print(chunk)

    for chunk_path in tqdm(chunks, desc="Reading dataset with pandas"):
        if verbose:
            print('Reading chunk:', chunk_path)
        chunk = pd.read_parquet(chunk_path, columns=columns)
        res.append(chunk)

    return pd.concat(res).reset_index(drop=True)

def prepare_transactions_dataset(
    path_to_dataset: str,
    num_parts_to_preprocess_at_once: int = 1,
    num_parts_total: int = 50,
    save_to_path: str = None,
    verbose: bool = False,
    columns: Optional[List[str]] = None
) -> pd.DataFrame:
    """
    Возвращает исходный pd.DataFrame с признаками из которых нужно собрать
    учебный датасет.

    Args:
        path_to_dataset : путь до датасета с партициями
        num_parts_to_preprocess_at_once : количество партиций, 
            которые будут одновременно держаться и обрабатываться в памяти
        num_parts_total : общее количество партиций, которые нужно обработать
        save_to_path : путь до папки для сохранения обработанных блоков в .parquet-формате; 
            если None, сохранение не происходит
        verbose : логировать каждую обрабатываемую часть данных
        columns : список колонок, которые нужно оставить

    Returns:
        pd.DataFrame : датафрейм с объединёнными данными
    """
    preprocessed_frames = []

    for step in tqdm(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                     desc="Transforming transactions data"):
        transactions_frame = read_parquet_dataset_from_local(
            path_to_dataset,
            start_from=step,
            num_parts_to_read=num_parts_to_preprocess_at_once,
            verbose=verbose,
            columns=columns
        )

       # Записываем подготовленные данные в файл
        if save_to_path:
            block_as_str = str(step)
            if len(block_as_str) == 1:
                block_as_str = '00' + block_as_str
            else:
                block_as_str = '0' + block_as_str
            transactions_frame.to_parquet(os.path.join(save_to_path, f'processed_chunk_{block_as_str}.parquet'))

        preprocessed_frames.append(transactions_frame)
    
    return pd.concat(preprocessed_frames)

In [20]:
"""
Собираем исходный датасет из parquet файлов,  
скачиваем только необходимые колонки
"""
# Путь до данных в проекте
path = 'train_data/'

data = prepare_transactions_dataset(
    path,
    num_parts_to_preprocess_at_once=1,
    num_parts_total=12,
    save_to_path='train_data/',
    columns=needed_columns) 

# Загружаем датасет с целевой переменной
target = pd.read_csv('train_target.csv')

# Делим датасет с целевой переменной на train/test части
y_train, y_test  = train_test_split(target, train_size=0.8, random_state=0, stratify=target.flag)

# Забираем наборы id из train/test
train_id = y_train['id'].values
test_id = y_test['id'].values

# На основе наборов id делим исходный датасет на train/test части
X_train = data.set_index('id').loc[train_id].reset_index()
X_test = data.set_index('id').loc[test_id].reset_index()

# Сбросим индексы для приведения к единому виду с X_train/X_test 
y_train = y_train.reset_index(drop=True)['flag']
y_test = y_test.reset_index(drop=True)['flag']

X_train.shape, X_test.shape, y_train.shape, y_test.shape

Transforming transactions data:   0%|          | 0/12 [00:00<?, ?it/s]

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

...

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

Reading dataset with pandas:   0%|          | 0/1 [00:00<?, ?it/s]

((20931476, 52), (5231241, 52), (2400000,), (600000,))

In [21]:
# Сохраним разделённые данные
X_train.to_csv('pipeline/X_train.csv', index=False)
X_test.to_csv('pipeline/X_test.csv', index=False)
y_train.to_csv('pipeline/y_train.csv', index=False)
y_test.to_csv('pipeline/y_test.csv', index=False)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20931476, 52), (5231241, 52), (2400000,), (600000,))

# Hyperparameters and weights for ensemble models

In [3]:
# Соберём список словарей с гиперпараметрами моделей
params_list = []
for i in range(5):
    model = CatBoostClassifier()
    model.load_model(f'catboost_models/fold_{i}_сonservative_model.bin')
    params_list.append(model.get_params())

with open('catboost_models/best_сonservative_params.pkl', 'rb') as file:
    final_model_params = pickle.load(file)
    
params_list.append(final_model_params['best_all_folds_params'])
params_list

[{'verbose': 0,
  'use_best_model': True,
  'random_seed': 0,
  'grow_policy': 'SymmetricTree',
  'border_count': 113,
  'min_data_in_leaf': 5,
  'random_strength': 8.209932299,
  'learning_rate': 0.03625476076,
  'iterations': 3000,
  'l2_leaf_reg': 8.005778243,
  'boosting_type': 'Plain',
  'od_wait': 100,
  'depth': 4,
  'subsample': 0.9882297325,
  'bagging_temperature': 0.09710127579,
  'rsm': 0.7343256008,
  'eval_metric': 'AUC',
  'loss_function': 'Logloss',
  'auto_class_weights': 'Balanced'},
 {'verbose': 0,
  'use_best_model': True,
  'random_seed': 0,
  'grow_policy': 'SymmetricTree',
  'border_count': 113,
  'min_data_in_leaf': 5,
  'random_strength': 8.209932299,
  'learning_rate': 0.03625476076,
  'iterations': 3000,
  'l2_leaf_reg': 8.005778243,
  'boosting_type': 'Plain',
  'od_wait': 100,
  'depth': 4,
  'subsample': 0.9882297325,
  'bagging_temperature': 0.09710127579,
  'rsm': 0.7343256008,
  'eval_metric': 'AUC',
  'loss_function': 'Logloss',
  'auto_class_weights':

In [4]:
# Собираем список весов моделей
with open('catboost_models/сonservative_models_weights.pkl', 'rb') as file:
    weights_list = pickle.load(file)

weights_list = weights_list['val_auc_list']
weights_list

[0.7576036850511159,
 0.7554545982995526,
 0.7532810994057619,
 0.7546988571803108,
 0.7524269260453276,
 0.7546930331964138]

# Pipeline

In [5]:
# Загружаем исходные разделённые данные 
X_train = pd.read_csv('pipeline/X_train.csv')
X_test = pd.read_csv('pipeline/X_test.csv')
y_train = pd.read_csv('pipeline/y_train.csv')
y_test = pd.read_csv('pipeline/y_test.csv')
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20931476, 52), (5231241, 52), (2400000, 1), (600000, 1))

In [6]:
# DATA PREPROCESSING FUNCTIONS

def convert_all_to_numeric_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Преобразует типы всех колоноки в числовые 
    с заменой ошибок на NaN (errors='coerce').

    Args:
        df : Исходный DataFrame, содержащий колонки 'id' и 'rn'.

    Returns:
        pandas.DataFrame : Копия исходного DataFrame 
        где все колонки приведены к числовому типу.
    """
    logger.info('FUNCTION convert_all_to_numeric_pipeline')
    
    # Копируем датасет чтобы не изменять оригинал.
    df = df.copy()
    
    return df.apply(lambda col: pd.to_numeric(col, errors='coerce'))


def convert_all_to_int_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Преобразует все колонки DataFrame к целочисленному типу.

    Args:
        df : Исходный DataFrame с числовыми значениями.

    Returns:
        pd.DataFrame : DataFrame, где все колонки приведены к типу int.
    """
    logger.info('FUNCTION convert_all_to_int_pipeline')
    
    return df.astype(int)

def drop_duplicates_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Удаляет дубликаты строк из DataFrame.

    Args:
        df : Исходный DataFrame.

    Returns:
        pd.DataFrame : DataFrame без дублирующихся строк.
    """
    
    logger.info('FUNCTION drop_duplicates_pipeline')
    
    return df.drop_duplicates()

In [7]:
# FEATURE ENGENERING FUNCTIONS

def rn_max_feature_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Добавляет в DataFrame новую колонку 'rn_max' — максимальное 
    значение 'rn' для каждой группы 'id'.

    Args:
        df : Исходный DataFrame, содержащий колонки 'id' и 'rn'.

    Returns:
        pandas.DataFrame : Копия исходного DataFrame с добавленной колонкой 'rn_max'.
    """
    logger.info('FUNCTION rn_max_feature_pipeline')

    """
    Для каждой строки определяем максимальное значение 'rn' среди всех строк с тем же 'id'
    Метод transform('max') возвращает Series длины исходного DataFrame, где для каждой строки
    указано максимальное значение 'rn' в её группе 'id'.
    """
    df['rn_max'] = df.groupby('id')['rn'].transform('max')

    return df

def enc_paym_transcoding_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """ 
    Прекодирует признаки enc_paym_features к единому виду с диапазоном значений {0, 1, 2, 3}.
    Для каждого столбца enc_paym_0, enc_paym_1, ..., enc_paym_24, 
    если в значениях встречается 4, происходит замена:
        1 -> 0
        2 -> 1
        3 -> 2
        4 -> 3

    Args:
        df : Исходный DataFrame с колонками 'enc_paym_0' ... 'enc_paym_24'.

    Returns:
    pandas.DataFrame : Копия DataFrame с перекодированными признаками.
    """
    logger.info('FUNCTION enc_paym_transcoding_pipeline ')
    
    # Список колонок для перекодировки
    columns = [f'enc_paym_{i}' for i in range(25)]
    
    for col in columns:
        # Проверяем, есть ли значение 4 в колонке
        if 4 in df[col].unique():
            # Заменяем значения согласно маппингу
            df.loc[:, col] = df[col].replace({1: 0, 2: 1, 3: 2, 4: 3})
            
    return df

def definite_value_proportion_features_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Создаёт и добавляет в датафрейм новые частотные признаки 
    на основе заданных значений исходных признаков.
    
    Для каждого столбца и каждого указанного значения в словаре функция создаёт новые признаки, 
    отражающие долю записей с этим значением относительно общего количества 
    кредитов (rn_max) для каждого id.
    
    Args:
        df : Исходный DataFrame, содержащий необходимые признаки и колонку 'rn_max'.
    
    Returns:
        pandas.DataFrame : Копия исходного DataFrame с добавленными частотными признаками.
    """
    logger.info('FUNCTION definite_value_proportion_features_pipeline')
    
    """
    Создадим словарь где для каждого признака перечислены значения,
    по которым считаем долю.
    """
    features_dictionary = {
        'enc_loans_account_holder_type': [4],
        'pre_pterm': [6, 3],
        'is_zero_loans530': [1],
        'enc_paym_0': [1],
        'pre_loans_credit_cost_rate': [6, 11, 4],
        'pre_loans_next_pay_summ': [5, 0],
        'is_zero_over2limit': [1],
        'pre_loans_outstanding': [1, 5],
        'pre_util': [3, 6],
        'pre_till_pclose': [10, 7],
        'is_zero_loans5': [1],
        'pre_since_confirmed': [4, 7],
        'pre_loans_credit_limit': [2, 15, 18],
        'pre_over2limit': [17],
        'pre_till_fclose': [4, 3, 1],
        'enc_loans_credit_status': [5],
        'pre_since_opened': [12, 8, 19],
        'enc_paym_24': [1],
        'pre_loans_max_overdue_sum': [1],
        'enc_loans_credit_type': [0, 2],
        'pre_fterm': [7, 3],
        'is_zero_loans3060': [1],
        'is_zero_loans6090': [1],
        'is_zero_loans90': [1]
    }   

    # Итерируем по ключам
    for col in  features_dictionary.keys():
        logger.info('Original feature %s', col)
        logger.info('New features')

        # Итерируем по значениям
        for value in features_dictionary[col]:
            new_column = f'{col}_prop_{value}'
            logger.info(new_column)                     

            """
            Создаём булевую маску: True, если значение в col равно value,
            иначе False.
            """
            mask = (df[col] == value)
            """
            Для каждой строки вычисляем количество совпадений value 
            по id (transform('sum')) и делим на общее количество кредитов 
            по id (rn_max), чтобы получить долю.
            """
            df[new_column] = mask.groupby(df['id']).transform('sum') / df['rn_max']        

    return df

def from_is_zero_prop_1_create_sum_prop_1_feature_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Вычисляет среднее значение признаков is_zero_*_prop_1 по строкам и добавляет 
    новый признак 'is_zero_sum_prop_1' в DataFrame.

    Args:
        df :  Исходный DataFrame с признаками is_zero_*_prop_1.

    Returns:
        pandas.DataFrame : Копия DataFrame с добавленным признаком 'is_zero_sum_prop_1'.
    """
    logger.info('FUNCTION from_is_zero_prop_1_create_sum_prop_1_feature_pipeline')

    columns = [
        'is_zero_loans5_prop_1',
        'is_zero_loans530_prop_1',
        'is_zero_loans3060_prop_1',
        'is_zero_loans6090_prop_1',
        'is_zero_loans90_prop_1'
    ]

    df['is_zero_sum_prop_1'] = df[columns].sum(axis=1) / 5

    return df

def mean_value_frequency_feature_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Cоздаёт новые агрегированные признаки,
    отражающий среднюю частоту (относительную встречаемость) значений 
    заданных столбцов columns_list датафрейма для каждого уникального id.
    Результат добавляется в  датафрейм 
    с нормировкой на количество записей (rn_max) для каждого id.
    
    Args:
        df :  Исходный DataFrame с признаками из columns_list.
    
    Returns:
        pandas.DataFrame :  Копия DataFrame с добавленным новым столбцом {column}_mean_freq,
        содержащим нормированное агрегированное значение средней 
        частоты значений column для каждого id.
    """
    logger.info('FUNCTION mean_value_frequency_feature_pipeline')
        
    # Список столбцов, для которых считаем среднюю частоту значений
    columns_list = [
        'pre_util',
        'pre_loans_credit_limit',
        'pre_since_opened',
        'pre_loans_credit_cost_rate',
        'enc_loans_credit_type',
        'pre_loans_next_pay_summ',
        'pre_since_confirmed',
        'pre_pterm',
        'enc_paym_0',
        'enc_loans_account_holder_type',
        'pre_loans530',
        'enc_paym_8',
        'pre_loans5',
        'enc_paym_10',
        'enc_loans_account_cur',
        'enc_paym_9'
    ]

    logger.info('New features')
    
    for col in columns_list:
        new_column = f'{col}_mean_freq'
        logger.info(new_column)
        
        # Вычисляем относительную частоту каждого уникального значения в столбце
        bin_freq = df[col].value_counts(normalize=True).to_dict()
        
        # Создаём Series с частотами значений для каждой строки
        freq_series = df[col].map(bin_freq)
        """
        Для каждой строки считаем сумму частот значений (freq_series) по группе 'id'.
        Делим эту сумму на общее количество записей по id (rn_max),
        чтобы получить нормированную среднюю частоту встречаемости значений
        признака для данного id.
        Результат сохраняем в новый столбец new_column.
        """
        df[new_column] = freq_series.groupby(df['id']).transform('sum') / df['rn_max']

    return df

def enc_paym_norm_group_sum_diff_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """ 
    Генерирует признаки разницы между средними количествами различных статусов платежей 
    по кредитам за разные временные промежутки.

    Основная цель функции — создать итоговые признаки:
        - 'enc_paym_avg_0_1_this_year_diff'
        - 'enc_paym_avg_1_2_all_diff'
        - 'enc_paym_avg_0_years_diff'

    Для их расчёта временно создаются промежуточные агрегированные признаки среднего 
    количества статусов платежей по id и периоду 
    (например, 'enc_paym_avg_0_this_year'), 
    которые впоследствии удаляются из итогового датасета.

    Args:
        df :  Исходный DataFrame с признаками из columns_list.
    
    Returns:
        pandas.DataFrame :  Копия DataFrame с добавленными итоговыми признаками 
        разницы между средними количествами статусов платежей по различным периодам.
    """

    logger.info('FUNCTION enc_paym_norm_group_sum_diff_pipeline')
    logger.info('New features')

    # Создаём временный датафрейм со столбцом id из df
    df_buff = pd.DataFrame(data = df['id'], columns = ['id'])
    
    # Временной промежуток 'all' — все периоды
    time_span = 'all'
    columns = [f'enc_paym_{i}' for i in range(25)]

    # Для статусов платежей по кредитам 1 и 2
    for i in range(1, 3):
        new_col = f'enc_paym_avg_{i}_{time_span}'
        logger.info(new_col)
        
        # Считаем количество статуса i по всем столбцам за период 
        df_buff[new_col] = np.sum(
            [df[col] == i for col in columns],
            axis=0
        )
        """
        Суммируем значения признака new_col по всем строкам с одинаковым id,
        затем делим на количество записей по этому id (rn_max),
        чтобы получить среднее количество появлений статуса для каждой строки.
        Cохраняем результат в новый столбец DataFrame с именем new_col.
        """
        df[new_col] = (
            df_buff[new_col].groupby(df_buff['id']).transform('sum') 
            / df['rn_max']
        )
        
    # Временной промежуток 'this_year' — первые 12 месяцев
    time_span = 'this_year'
    columns = [f'enc_paym_{i}' for i in range(12)]

    # Для статусов платежей по кредитам 0 и 1
    for i in range(2):
        new_col = f'enc_paym_avg_{i}_{time_span}'
        logger.info(new_col)
        
        # Считаем количество статуса i по всем столбцам за период 
        df_buff[new_col] = np.sum(
            [df[col] == i for col in columns],
            axis=0
        )
        """
        Суммируем значения признака new_col по всем строкам с одинаковым id,
        затем делим на количество записей по этому id (rn_max),
        чтобы получить среднее количество появлений статуса для каждой строки.
        Cохраняем результат в новый столбец DataFrame с именем new_col.
        """
        df[new_col] = (
            df_buff[new_col].groupby(df_buff['id']).transform('sum')
            / df['rn_max']
        )
        
    # Временной промежуток 'last_year' — месяцы с 12 по 24
    time_span = 'last_year'
    columns = [f'enc_paym_{i}' for i in range(12, 25)]
    
    """
    Статус платежей  0.
    (Оставим цикл для единообразия кода)
    """
    for i in [0]:
        new_col = f'enc_paym_avg_{i}_{time_span}'
        logger.info(new_col)
        
        # Считаем количество статуса i по всем столбцам за период 
        df_buff[new_col] = np.sum(
            [df[old_col] == i for old_col in columns],
            axis=0
        )
        """
        Суммируем значения признака new_col по всем строкам с одинаковым id,
        затем делим на количество записей по этому id (rn_max),
        чтобы получить среднее количество появлений статуса для каждой строки.
        Cохраняем результат в новый столбец DataFrame с именем new_col.
        """
        df[new_col] = (
            df_buff[new_col].groupby(df_buff['id']).transform('sum')
            / df['rn_max']
        )

    # Создаём фичи разницы 
    df['enc_paym_avg_0_1_this_year_diff'] = (
            df['enc_paym_avg_0_this_year'] - 
            df['enc_paym_avg_1_this_year']
    )

    df['enc_paym_avg_1_2_all_diff'] = (
            df['enc_paym_avg_1_all'] - 
            df['enc_paym_avg_2_all']
    )

    df['enc_paym_avg_0_years_diff'] = (
            df['enc_paym_avg_0_this_year'] - 
            df['enc_paym_avg_0_last_year']
    )
    

    logger.info("""\
New difference columns
enc_paym_avg_0_1_this_year_diff
enc_paym_avg_1_2_all_diff
enc_paym_avg_0_years_diff
"""
               )

    return df

def pre_since_opened_sum_mean_repeated_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Cоздаёт признак, отражающий пропорцию повторяющихся значений 'pre_since_opened'
    для каждого 'id'.

    Логика работы:
    - Подсчитывает количество появлений каждого значения 'pre_since_opened' для каждого 'id'.
    - Выделяет только повторяющиеся значения (где количество > 1) и вычитает 1,
      чтобы не считать первое появление.
    - Суммирует количество повторов по всем значениям 'pre_since_opened' для каждого 'id'.
    - Добавляет отсутствующие 'id' с нулевыми значениями повторов.
    - Добавляет новый признак 'pre_since_opened_repeated_prop' в df_to_update,
      нормируя сумму повторов на количество записей 'rn_max' для каждого 'id'.

    Args:
        df :  Исходный DataFrame с признаками  'pre_since_opened', 'id' и 'rn_max'.
    
    Returns:
        pandas.DataFrame :  Копия DataFrame с 
        добавленным признаком 'pre_since_opened_repeated_prop'.
    """
    logger.info('FUNCTION pre_since_opened_sum_mean_repeated_pipeline')
    
    # Считаем количество каждого значения 'pre_since_opened' для каждого 'id'
    counts = df.groupby(['id', 'pre_since_opened']).size()
    
    """
    Оставляем только повторяющиеся значения (количество > 1), 
    вычитаем первое появление.
    """
    repeated_pre_since_opened = counts[counts > 1] - 1

    # Суммируем количество повторов по каждому 'id'
    sum_repeated = repeated_pre_since_opened.groupby('id').sum()
    
    # Добавляем отсутствующие 'id' с нулевыми значениями повторов
    all_sum_repeated = sum_repeated.reindex(df['id'].unique(), fill_value=0)
    
    # Переименовываем Series для дальнейшего слияния
    all_sum_repeated = all_sum_repeated.rename('pre_since_opened_repeated_prop')

    # Объединяем с исходным DataFrame по 'id'
    df = df.merge(all_sum_repeated, on='id', how='left')

    # Нормируем сумму повторов на количество записей 'rn_max' для каждого 'id'
    df['pre_since_opened_repeated_prop'] = (
        df['pre_since_opened_repeated_prop'] / df['rn_max']
    )

    return df


def drop_columns_drop_duplicates_pipeline(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Удаляет исходные и временные признаки из DataFrame,
    а также удаляет дубликаты по столбцу 'id', оставляя только первую запись.
    После удаления дубликатов столбец 'id' также удаляется.

    Args:
        df : Исходный DataFrame.

    Returns:
        pd.DataFrame : Копия DataFrame без указанных столбцов и дубликатов по 'id'.
    """

    logger.info('FUNCTION drop_columns_drop_duplicates_pipeline')
    # Список столбцов на удаление
    columns = [
        'rn',
        'pre_since_opened',
        'pre_since_confirmed',
        'pre_pterm',
        'pre_fterm',
        'pre_till_pclose',
        'pre_till_fclose',
        'pre_loans_credit_limit',
        'pre_loans_next_pay_summ',
        'pre_loans_outstanding',
        'pre_loans_max_overdue_sum',
        'pre_loans_credit_cost_rate',
        'pre_loans5',
        'pre_loans530',
        'is_zero_loans5',
        'is_zero_loans530',
        'pre_util',
        'pre_over2limit',
        'is_zero_over2limit',
        'enc_paym_0',
        'enc_paym_1',
        'enc_paym_2',
        'enc_paym_8',
        'enc_paym_9',
        'enc_paym_10',
        'enc_paym_24',
        'enc_loans_account_holder_type',
        'enc_loans_credit_status',
        'enc_loans_credit_type',
        'enc_loans_account_cur',
        'is_zero_loans3060',
        'is_zero_loans6090',
        'is_zero_loans90',
        'enc_paym_3',
        'enc_paym_4',
        'enc_paym_5',
        'enc_paym_6',
        'enc_paym_7',
        'enc_paym_11',
        'enc_paym_12',
        'enc_paym_13',
        'enc_paym_14',
        'enc_paym_15',
        'enc_paym_16',
        'enc_paym_17',
        'enc_paym_18',
        'enc_paym_19',
        'enc_paym_20',
        'enc_paym_21',
        'enc_paym_22',
        'enc_paym_23',
        'enc_paym_avg_1_all',
        'enc_paym_avg_2_all',
        'enc_paym_avg_0_this_year',
        'enc_paym_avg_1_this_year',
        'enc_paym_avg_0_last_year',
        'is_zero_loans3060_prop_1',
        'is_zero_loans6090_prop_1',
        'is_zero_loans90_prop_1'
    ]
    
    df = df.drop(columns, axis=1)
    
    """
    Удаляем дубликаты по столбцу 'id', оставляя первую запись
    и сбрасываем индекс.
    """
    df = df.drop_duplicates(subset=['id'], keep='first').reset_index(drop=True)
    
    # Удаляем столбец 'id', так как он больше не нужен
    df = df.drop('id', axis=1)
    
    return df

In [8]:
# CLASSIFIER

class CatBoostEnsembleClassifier(BaseEstimator, ClassifierMixin):
    """
    Ансамблевый классификатор на основе CatBoost, обучающий 5 моделей 
    на разных разбиениях данных и финальную модель на полном наборе данных.
    Чтобы повысить устойчивость и качество предсказаний 
    за счёт усреднения результатов.

    Особенности:
        - Используется StratifiedKFold для разбиения данных на 5 фолдов.
        - Для каждого фолда обучается отдельная модель на тренировочной части.
        - Все 5 моделей сохраняются для последующего усреднения предсказаний.
        - Дополнительно обучается финальная модель на полном наборе данных.
        - Предсказания объединяются с учётом весов моделей.

    Args:
        params_list (list of dict): Список параметров для каждой из 6 моделей.
        weights_list (list of float): Веса для усреднения предсказаний моделей
            представляющие собой auc_score шести моделей. 
            Это позволяет отдавать большее значение более качественным моделям.
            
    Attributes:
        models_ (list): Список обученных моделей CatBoostClassifier 
            после вызова fit.

    Methods:
        fit(X, y): Обучает ансамбль моделей и сохраняет их.
        fit_transform(X, y): Обучает модели и возвращает X без изменений 
            (для совместимости с пайплайнами).
        transform(X): Возвращает X без изменений 
            (для совместимости с пайплайнами).
        predict_proba(X): Возвращает взвешенное усреднённое предсказание 
            вероятностей положительного класса.
        predict(X): Возвращает бинарные предсказания с порогом 0.4964.
    """
    def __init__(self, params_list, weights_list):
        self.params_list = params_list
        self.weights_list = weights_list

    def fit(self, X, y):
        """
        Обучает ансамбль моделей на разных фолдах и финальную модель 
        на полном наборе данных.

        Args:
            X (pd.DataFrame): Признаки.
            y (pd.Series или np.array): Целевой признак.

        Returns:
            self : Обученный объект классификатора с атрибутом models_, 
              содержащим список обученных моделей.
        """
        logger.info('CLASSIFIER fit')
        
        self.models_ = []
        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

        # Обучаем 5 моделей на разных фолдах
        for i, (train_idx, val_idx) in enumerate(kf.split(X, y)):
            
            # Выводим номер фолда в лог
            logger.info('Fit fold %s', i)
            
            X_train = X.iloc[train_idx]
            y_train = y.iloc[train_idx]
            X_val = X.iloc[val_idx]
            y_val = y.iloc[val_idx]
            
            train_pool = Pool(data=X_train, label=y_train, cat_features=[])
            val_pool = Pool(data=X_val, label=y_val, cat_features=[])

            """
            Для обучения используются валидационные подвыборки, 
            для остановки обучения вместо ГП early_stopping_rounds
            в params передаётся od_wait.
            """
            params = self.params_list[i]

            # Выводим гипер параметры в лог
            logger.info('Params for fold %s', params)
            
            model = CatBoostClassifier(
                **params
            )
            model.fit(
                train_pool,
                eval_set=val_pool
            )
            self.models_.append(model)
            
        # Обучаем финальную модель на полном наборе данных
        logger.info('Fit final model')
        train_pool = Pool(data=X, label=y, cat_features=[])
        
        params = self.params_list[5]
        # Выводим гипер параметры в лог
        logger.info('Params for final model  %s', params)
        
        model = CatBoostClassifier(
            **params
        )
        model.fit(
            train_pool
        )
        self.models_.append(model)
        return self

    def fit_transform(self, X, y=None):
        logger.info('CLASSIFIER fit_transform')
        # Обучаем классификатор
        self.fit(X, y)
        # Возвращаем X без изменений
        return X

    def transform(self, X):
        logger.info('CLASSIFIER transform')
        # Возвращаем X без изменений
        return X

    def predict_proba(self, X):
        """
        Предсказывает вероятности классов, 
        усреднённые по всем моделям с учётом весов.

        Args:
            X (pd.DataFrame): Матрица признаков.

        Returns:
            np.ndarray: Массив вероятностей для классов 0 и 1,
            размерностью (n_samples, 2).
        """
        logger.info('CLASSIFIER predict_proba')
        
        preds = []
        for model, weight in zip(self.models_, self.weights_list):
            pred = model.predict_proba(X)[:, 1]
            preds.append(pred * weight)
        mean_pred = np.sum(preds, axis=0) / np.sum(self.weights_list)
        return np.vstack([1 - mean_pred, mean_pred]).T

    def predict(self, X):
        """
        Предсказывает классы на основе вероятностей с порогом 0.4964,
        при котором разница между TPR и FPR максимальна.

        Args:
            X (pd.DataFrame): Матрица признаков.

        Returns:
            np.ndarray: Массив предсказанных классов (0 или 1).
        """
        logger.info('CLASSIFIER predict')
        proba = self.predict_proba(X)[:, 1]
        return (proba >= 0.4964).astype(int)

In [9]:
"""
Из-за большого размера датасета вычисление медиан признаков занимает
большой объём памяти, что приводит к падению ядра ноутбука. Поэтому применим
кастомный imputer и будем расчитывать медианы на 10% процентах датасета.
Оценки медианы будут приближены к реальным медианам, но не совпадать с ними.
Такое решение это компромис между точностью и производительностью.
Например для признака id погрешность между реальной медианой и оценочной 
составила около 0.05%.
"""
class SampleMedianImputer(BaseEstimator, TransformerMixin):
    def __init__(self, sample_frac=0.1):
        # Доля выборки для вычисления медиан
        self.sample_frac = sample_frac
        # Атрибут для хранения медиан
        self.medians_ = None

    def fit(self, X, y=None):
        # Создаём подвыборку датасета, random_state для воспроизводимости
        sample = X.sample(frac=self.sample_frac, random_state=0)
        # Вычисляем и сохраняем медианы
        self.medians_ = sample.median()
        return self

    def transform(self, X):
        # Заполняем пропуски медианами
        return X.fillna(self.medians_)

In [10]:
# Создаём SampleMedianImputer, долю выборки оставляем равной 0.1
imputer = SampleMedianImputer(sample_frac=0.1)

# Создаём паплайн препроцессинга
preprocessing_pipe = Pipeline([
    ('to_numeric', FunctionTransformer(convert_all_to_numeric_pipeline)), 
    ('imputer', imputer),
    ('to_int', FunctionTransformer(convert_all_to_int_pipeline)),
    ('drop_duplicates', FunctionTransformer(drop_duplicates_pipeline))
])

# Создаём основной пайплайн
main_pipe = Pipeline(
    [
        (
            'preprocessing',
            preprocessing_pipe
        ),
        (
            'create_rn_max_feature',
            FunctionTransformer(rn_max_feature_pipeline)
        ),
        (
            'enc_paym_transcoding', 
            FunctionTransformer(enc_paym_transcoding_pipeline)
        ),
        (
            'create_definite_value_proportion_features',
            FunctionTransformer(definite_value_proportion_features_pipeline)
        ),
        (
            'create_sum_prop_1_feature',
            FunctionTransformer(from_is_zero_prop_1_create_sum_prop_1_feature_pipeline)
        ),
        (
            'create_mean_value_frequency_feature',
            FunctionTransformer(mean_value_frequency_feature_pipeline)
        ),
        (
            'from_enc_paym_create_normalized_group_sum_features_then_diff_features',
            FunctionTransformer(enc_paym_norm_group_sum_diff_pipeline)
        ),
        (
            'from_pre_since_opened_create_pre_since_opened_sum_mean_repeated',
            FunctionTransformer(pre_since_opened_sum_mean_repeated_pipeline)
        ),
        (
            'drop_temporary_and_source_columns_drop_duplicates',
            FunctionTransformer(drop_columns_drop_duplicates_pipeline)
        ),
         (
            'classifier', 
            CatBoostEnsembleClassifier(params_list=params_list, weights_list=weights_list)
        )

    ]
)

In [11]:
# Обучим пайплайн
main_pipe.fit(X_train, y_train)

2025-07-03 13:41:52,502 - FUNCTION convert_all_to_numeric_pipeline
2025-07-03 13:43:37,741 - FUNCTION convert_all_to_int_pipeline
2025-07-03 13:44:04,273 - FUNCTION drop_duplicates_pipeline
2025-07-03 13:46:23,965 - FUNCTION rn_max_feature_pipeline
2025-07-03 13:46:28,110 - FUNCTION enc_paym_transcoding_pipeline 
2025-07-03 13:46:36,654 - FUNCTION definite_value_proportion_features_pipeline
2025-07-03 13:46:36,658 - Original feature enc_loans_account_holder_type
2025-07-03 13:46:36,666 - New features
2025-07-03 13:46:36,671 - enc_loans_account_holder_type_prop_4
2025-07-03 13:46:39,641 - Original feature pre_pterm
2025-07-03 13:46:39,644 - New features
2025-07-03 13:46:39,645 - pre_pterm_prop_6
2025-07-03 13:46:42,631 - pre_pterm_prop_3
2025-07-03 13:46:45,285 - Original feature is_zero_loans530
2025-07-03 13:46:45,286 - New features
2025-07-03 13:46:45,287 - is_zero_loans530_prop_1
2025-07-03 13:46:48,295 - Original feature enc_paym_0
2025-07-03 13:46:48,298 - New features
2025-07-03 

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('to_numeric',
                                  FunctionTransformer(func=<function convert_all_to_numeric_pipeline at 0x1434a1080>)),
                                 ('imputer', SampleMedianImputer()),
                                 ('to_int',
                                  FunctionTransformer(func=<function convert_all_to_int_pipeline at 0x1434a0f40>)),
                                 ('drop_duplicates',
                                  FunctionTransformer(func=<function drop_duplicates_pipeline at 0x14...
                                                          'iterations': 2996,
                                                          'l2_leaf_reg': 8.005778242558318,
                                                          'learning_rate': 0.036254760756236626,
                                                          'min_data_in_leaf': 5,
                                                          'random_seed': 0,
                                                          'random_strength': 8.209932298658357,
                                                          'rsm': 0.7343256008238508,
                                                          'subsample': 0.9882297325066979,
                                                          'verbose': False}],
                                            weights_list=[0.7576036850511159,
                                                          0.7554545982995526,
                                                          0.7532810994057619,
                                                          0.7546988571803108,
                                                          0.7524269260453276,
                                                          0.7546930331964138]))])

In [12]:
# Сохраним обученный пайплайн в файл
with open('pipeline/trained_pipeline.pkl', 'wb') as file:
    pickle.dump(main_pipe, file)

In [13]:
# Загрузим обученный пайплайн
with open('pipeline/trained_pipeline.pkl', 'rb') as file:
   main_pipe = pickle.load(file)
main_pipe

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('to_numeric',
                                  FunctionTransformer(func=<function convert_all_to_numeric_pipeline at 0x1434a1080>)),
                                 ('imputer', SampleMedianImputer()),
                                 ('to_int',
                                  FunctionTransformer(func=<function convert_all_to_int_pipeline at 0x1434a0f40>)),
                                 ('drop_duplicates',
                                  FunctionTransformer(func=<function drop_duplicates_pipeline at 0x14...
                                                          'iterations': 2996,
                                                          'l2_leaf_reg': 8.005778242558318,
                                                          'learning_rate': 0.036254760756236626,
                                                          'min_data_in_leaf': 5,
                                                          'random_seed': 0,
                                                          'random_strength': 8.209932298658357,
                                                          'rsm': 0.7343256008238508,
                                                          'subsample': 0.9882297325066979,
                                                          'verbose': False}],
                                            weights_list=[0.7576036850511159,
                                                          0.7554545982995526,
                                                          0.7532810994057619,
                                                          0.7546988571803108,
                                                          0.7524269260453276,
                                                          0.7546930331964138]))])

In [14]:
# Предскажем вероятности классов
pred_proba = main_pipe.predict_proba(X_test)
pred_proba

2025-07-03 15:11:45,649 - FUNCTION convert_all_to_numeric_pipeline
2025-07-03 15:11:55,508 - FUNCTION convert_all_to_int_pipeline
2025-07-03 15:11:56,939 - FUNCTION drop_duplicates_pipeline
2025-07-03 15:12:08,998 - FUNCTION rn_max_feature_pipeline
2025-07-03 15:12:09,623 - FUNCTION enc_paym_transcoding_pipeline 
2025-07-03 15:12:10,831 - FUNCTION definite_value_proportion_features_pipeline
2025-07-03 15:12:10,832 - Original feature enc_loans_account_holder_type
2025-07-03 15:12:10,834 - New features
2025-07-03 15:12:10,835 - enc_loans_account_holder_type_prop_4
2025-07-03 15:12:11,250 - Original feature pre_pterm
2025-07-03 15:12:11,251 - New features
2025-07-03 15:12:11,251 - pre_pterm_prop_6
2025-07-03 15:12:11,785 - pre_pterm_prop_3
2025-07-03 15:12:12,212 - Original feature is_zero_loans530
2025-07-03 15:12:12,213 - New features
2025-07-03 15:12:12,214 - is_zero_loans530_prop_1
2025-07-03 15:12:12,656 - Original feature enc_paym_0
2025-07-03 15:12:12,657 - New features
2025-07-03 

array([[0.94601576, 0.05398424],
       [0.53785636, 0.46214364],
       [0.65246789, 0.34753211],
       ...,
       [0.56149362, 0.43850638],
       [0.83209011, 0.16790989],
       [0.31052491, 0.68947509]])

In [15]:
# Вычислим целевую метрику
roc_auc_score(y_test, pred_proba[:,1])

0.7572266329942543

Метрика roc_auc_score получилась практически такой же как и в исследовательском ноутбуке (0.7572338055768848),
думаю разницу можно объяснить погрешностью вычислений, к тому же схемы сбора тренировочных датасетов немного отличается.
В исследовательской части мы набирали признаки  из исходного датасета в датасет с таргетом размером (2400000, 1) а в ноутбуке пайплайна сразу в исходный датасет  размером (20931476, N) и после удаляли дубликаты строк чтобы привести в соответствие с размером таргет датасета.


In [17]:
# Отключим логирование и предскажем класс 1
pred = main_pipe.predict(X_test)
pred

array([0, 0, 0, ..., 0, 0, 1])

In [19]:
# Проверим метод трансформера также без логирования
X_test_transformed = main_pipe.transform(X_test)
X_test_transformed

,rn_max,enc_loans_account_holder_type_prop_4,pre_pterm_prop_6,pre_pterm_prop_3,is_zero_loans530_prop_1,enc_paym_0_prop_1,pre_loans_credit_cost_rate_prop_6,pre_loans_credit_cost_rate_prop_11,pre_loans_credit_cost_rate_prop_4,pre_loans_next_pay_summ_prop_5,...,pre_loans530_mean_freq,enc_paym_8_mean_freq,pre_loans5_mean_freq,enc_paym_10_mean_freq,enc_loans_account_cur_mean_freq,enc_paym_9_mean_freq,enc_paym_avg_0_1_this_year_diff,enc_paym_avg_1_2_all_diff,enc_paym_avg_0_years_diff,pre_since_opened_repeated_prop
0,10,0.0,0.200000,0.000000,1.000000,0.000000,0.000000,0.000000,0.600000,0.100000,...,0.976465,0.519372,0.994481,0.460134,0.997625,0.492167,11.000000,0.000000,3.000000,0.200000
1,8,0.0,0.000000,0.000000,0.875000,0.125000,0.000000,0.000000,0.125000,0.000000,...,0.976465,0.358095,0.994481,0.484894,0.997625,0.483197,5.625000,1.000000,3.125000,0.250000
2,24,0.0,0.041667,0.000000,0.833333,0.000000,0.000000,0.041667,0.041667,0.000000,...,0.976465,0.497436,0.994481,0.484894,0.997625,0.484131,8.625000,0.250000,5.458333,0.416667
3,10,0.0,0.100000,0.100000,0.900000,0.000000,0.000000,0.000000,0.900000,0.000000,...,0.976465,0.481767,0.994481,0.484894,0.997625,0.483197,7.500000,0.100000,2.600000,0.100000
4,18,0.0,0.000000,0.166667,0.888889,0.055556,0.277778,0.000000,0.111111,0.000000,...,0.976465,0.416355,0.994481,0.498649,0.997625,0.478213,4.833333,0.333333,4.388889,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,6,0.0,0.000000,0.000000,0.500000,0.166667,0.000000,0.166667,0.000000,0.166667,...,0.816428,0.414988,0.994481,0.424269,0.997625,0.483197,5.666667,1.333333,6.000000,0.166667
599996,3,0.0,0.000000,0.000000,0.666667,0.000000,0.000000,0.333333,0.000000,0.000000,...,0.976465,0.497436,0.994481,0.474577,0.997625,0.486934,8.000000,0.333333,2.000000,0.000000
599997,5,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.200000,...,0.976465,0.453562,0.994481,0.503464,0.997625,0.476469,5.600000,0.000000,3.000000,0.400000
599998,7,0.0,0.285714,0.000000,0.857143,0.000000,0.000000,0.142857,0.000000,0.285714,...,0.976465,0.475051,0.994481,0.489315,0.997625,0.481595,7.571429,0.000000,3.142857,0.285714


In [20]:
# Сохраним предсказания классов и их вероятностей в файлы
with open('pipeline/test_predict.pkl', 'wb') as file:
    pickle.dump(pred, file)

with open('pipeline/test_predict_proba.pkl', 'wb') as file:
    pickle.dump(pred_proba, file)